# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 8 2023 9:33AM,256900,15,9127019,"Brookfield Pharmaceuticals, LLC",Released
1,Feb 8 2023 9:33AM,256900,15,9127020,"Brookfield Pharmaceuticals, LLC",Released
2,Feb 8 2023 9:33AM,256900,15,9127021,"Brookfield Pharmaceuticals, LLC",Released
3,Feb 8 2023 9:33AM,256900,15,9127022,"Brookfield Pharmaceuticals, LLC",Released
4,Feb 8 2023 9:33AM,256900,15,9127023,"Brookfield Pharmaceuticals, LLC",Released
5,Feb 8 2023 9:33AM,256900,15,9127024,"Brookfield Pharmaceuticals, LLC",Released
6,Feb 8 2023 9:33AM,256900,15,9127025,"Brookfield Pharmaceuticals, LLC",Released
7,Feb 8 2023 9:33AM,256900,15,9127026,"Brookfield Pharmaceuticals, LLC",Released
8,Feb 8 2023 9:33AM,256900,15,9127027,"Brookfield Pharmaceuticals, LLC",Released
9,Feb 8 2023 9:33AM,256900,15,9127028,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
43,256894,Released,2
44,256897,Released,17
45,256898,Released,1
46,256899,Released,1
47,256900,Released,16


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256894,NaN,NaN,2.0
256897,NaN,NaN,17.0
256898,NaN,NaN,1.0
256899,NaN,NaN,1.0
256900,NaN,NaN,16.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256783,25.0,0.0,27.0
256805,0.0,0.0,1.0
256807,0.0,0.0,1.0
256808,0.0,0.0,1.0
256814,0.0,12.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256783,25,0,27
256805,0,0,1
256807,0,0,1
256808,0,0,1
256814,0,12,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256783,25,0,27
1,256805,0,0,1
2,256807,0,0,1
3,256808,0,0,1
4,256814,0,12,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256783,25,,27
1,256805,,,1
2,256807,,,1
3,256808,,,1
4,256814,,12,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 8 2023 9:33AM,256900,15,"Brookfield Pharmaceuticals, LLC"
16,Feb 8 2023 9:28AM,256899,10,"Citieffe, Inc."
17,Feb 8 2023 9:25AM,256898,22,"NBTY Global, Inc."
18,Feb 8 2023 9:15AM,256897,10,"Methapharm, Inc."
35,Feb 8 2023 9:09AM,256892,18,Innova Softgel LLC
36,Feb 8 2023 9:07AM,256890,10,ISDIN Corporation
58,Feb 8 2023 9:07AM,256888,10,ISDIN Corporation
109,Feb 8 2023 9:07AM,256887,10,ISDIN Corporation
136,Feb 8 2023 9:07AM,256894,10,"Methapharm, Inc."
138,Feb 8 2023 9:07AM,256891,16,Innova Softgel LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 8 2023 9:33AM,256900,15,"Brookfield Pharmaceuticals, LLC",,,16
1,Feb 8 2023 9:28AM,256899,10,"Citieffe, Inc.",,,1
2,Feb 8 2023 9:25AM,256898,22,"NBTY Global, Inc.",,,1
3,Feb 8 2023 9:15AM,256897,10,"Methapharm, Inc.",,,17
4,Feb 8 2023 9:09AM,256892,18,Innova Softgel LLC,,1,
5,Feb 8 2023 9:07AM,256890,10,ISDIN Corporation,,,22
6,Feb 8 2023 9:07AM,256888,10,ISDIN Corporation,,,51
7,Feb 8 2023 9:07AM,256887,10,ISDIN Corporation,,,27
8,Feb 8 2023 9:07AM,256894,10,"Methapharm, Inc.",,,2
9,Feb 8 2023 9:07AM,256891,16,Innova Softgel LLC,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 9:33AM,256900,15,"Brookfield Pharmaceuticals, LLC",16,,
1,Feb 8 2023 9:28AM,256899,10,"Citieffe, Inc.",1,,
2,Feb 8 2023 9:25AM,256898,22,"NBTY Global, Inc.",1,,
3,Feb 8 2023 9:15AM,256897,10,"Methapharm, Inc.",17,,
4,Feb 8 2023 9:09AM,256892,18,Innova Softgel LLC,,1,
5,Feb 8 2023 9:07AM,256890,10,ISDIN Corporation,22,,
6,Feb 8 2023 9:07AM,256888,10,ISDIN Corporation,51,,
7,Feb 8 2023 9:07AM,256887,10,ISDIN Corporation,27,,
8,Feb 8 2023 9:07AM,256894,10,"Methapharm, Inc.",2,,
9,Feb 8 2023 9:07AM,256891,16,Innova Softgel LLC,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 9:33AM,256900,15,"Brookfield Pharmaceuticals, LLC",16,,
1,Feb 8 2023 9:28AM,256899,10,"Citieffe, Inc.",1,,
2,Feb 8 2023 9:25AM,256898,22,"NBTY Global, Inc.",1,,
3,Feb 8 2023 9:15AM,256897,10,"Methapharm, Inc.",17,,
4,Feb 8 2023 9:09AM,256892,18,Innova Softgel LLC,,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 9:33AM,256900,15,"Brookfield Pharmaceuticals, LLC",16.0,NaN,NaN
1,Feb 8 2023 9:28AM,256899,10,"Citieffe, Inc.",1.0,NaN,NaN
2,Feb 8 2023 9:25AM,256898,22,"NBTY Global, Inc.",1.0,NaN,NaN
3,Feb 8 2023 9:15AM,256897,10,"Methapharm, Inc.",17.0,NaN,NaN
4,Feb 8 2023 9:09AM,256892,18,Innova Softgel LLC,NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 9:33AM,256900,15,"Brookfield Pharmaceuticals, LLC",16.0,0.0,0.0
1,Feb 8 2023 9:28AM,256899,10,"Citieffe, Inc.",1.0,0.0,0.0
2,Feb 8 2023 9:25AM,256898,22,"NBTY Global, Inc.",1.0,0.0,0.0
3,Feb 8 2023 9:15AM,256897,10,"Methapharm, Inc.",17.0,0.0,0.0
4,Feb 8 2023 9:09AM,256892,18,Innova Softgel LLC,0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4623578,176.0,23.0,1.0
15,513683,43.0,0.0,25.0
16,1284329,5.0,2.0,0.0
18,256892,0.0,1.0,0.0
19,1541133,30.0,17.0,0.0
21,1284170,5.0,0.0,0.0
22,256898,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4623578,176.0,23.0,1.0
1,15,513683,43.0,0.0,25.0
2,16,1284329,5.0,2.0,0.0
3,18,256892,0.0,1.0,0.0
4,19,1541133,30.0,17.0,0.0
5,21,1284170,5.0,0.0,0.0
6,22,256898,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,176.0,23.0,1.0
1,15,43.0,0.0,25.0
2,16,5.0,2.0,0.0
3,18,0.0,1.0,0.0
4,19,30.0,17.0,0.0
5,21,5.0,0.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,176.0
1,15,Released,43.0
2,16,Released,5.0
3,18,Released,0.0
4,19,Released,30.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,21,22
Status,,,,,,,
Completed,1.0,25.0,0.0,0.0,0.0,0.0,0.0
Executing,23.0,0.0,2.0,1.0,17.0,0.0,0.0
Released,176.0,43.0,5.0,0.0,30.0,5.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,21,22
0,Completed,1.0,25.0,0.0,0.0,0.0,0.0,0.0
1,Executing,23.0,0.0,2.0,1.0,17.0,0.0,0.0
2,Released,176.0,43.0,5.0,0.0,30.0,5.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,21,22
0,Completed,1.0,25.0,0.0,0.0,0.0,0.0,0.0
1,Executing,23.0,0.0,2.0,1.0,17.0,0.0,0.0
2,Released,176.0,43.0,5.0,0.0,30.0,5.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()